# Credit Card fraud prediction using original and custom model without smote

In [ ]:
!pip install opendatasets --quiet

In [ ]:
import opendatasets as od
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import seaborn as sns

In [ ]:
import numpy as np 
from numpy import log,dot,exp,shape
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
link = 'https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud'
od.download(link)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: royalalbert
Your Kaggle Key: ··········


100%|██████████| 66.0M/66.0M [00:00<00:00, 179MB/s]


In [ ]:
df = pd.read_csv(r'/content/creditcardfraud/creditcard.csv');df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
df['NormAmt'] = StandardScaler().fit_transform(df[['Amount']])
X = df[df.columns.to_list()[1:]].drop(['Class', 'Amount'], axis = 1)
y = df['Class']

In [ ]:
from sklearn.model_selection import train_test_split    # create a custom k-fold cross validation.. dont use train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25, random_state = 42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.15, random_state = 42)

In [ ]:
from tqdm import tqdm

class LogisticRegressionCustom:
    def __init__(self, lr, epochs):      
        self.lr = lr
        self.epochs = epochs
        self.weight = 0
        self.bias = 0
        self.loss = []
        self.accuracy_record = []
        self.weight_record = []
        self.bias_record = []
        self.epoch_record = []

    def get_clean_output(self, preds):
      threshold_prob = 0.5
      result = preds >= threshold_prob
      y_pred = np.zeros(result.shape[0])
      for i in range(len(y_pred)):
          if result[i] == True: 
              y_pred[i] = 1
          else:
              continue
                 
      return y_pred


    def _accuracy(self, target, predictions):
      accuracy = 0
      for i in range(len(predictions)):
        if (predictions[i] == target[i]):
          accuracy += 1

      return accuracy / len(target)

    # Sigmoid method
    def sigmoid(self, x, weight, bias):
        z = np.dot(x, weight) + bias
        return 1 / (1 + np.exp(-z))
     
    # method to calculate the binary cross entropy Loss
    def BCE_loss(self, preds, y):
        return (-y * np.log(preds) - (1 - y) * np.log(1 - preds)).mean()

    # method for calculating the gradients
    def gradient_descent(self, X, preds, target):
      d_loss_wrt_weight = (1 / X.shape[0]) * np.dot(X.T, (preds - target)) 
      d_loss_wrt_bias =  (1 / X.shape[0]) * np.sum(preds - target)
      return d_loss_wrt_weight, d_loss_wrt_bias
     
    def fit(self, X, y, X_val, y_val):
      self.weight = np.zeros(X.shape[1])
      # training set
      X = X.values
      y = y.values

      # Validation set
      X_val = X_val.values
      y_val = y_val.values

      # training
      for i in tqdm(range(self.epochs)):
        predictions = self.sigmoid(X, self.weight, self.bias)
        loss = self.BCE_loss(predictions, y)
        dW, dB = self.gradient_descent(X, predictions, y)
        
        # recording items
        self.loss.append(loss)
        # recording validation accuracy
        self.accuracy_record.append(self._accuracy(y_val, self.get_clean_output(self.sigmoid(X_val, self.weight, self.bias))))
        self.weight_record.append(self.weight)
        self.bias_record.append(self.bias)
        self.epoch_record.append(i)
 
        # Updating the weights
        self.weight -= self.lr * dW
        self.bias -= self.lr * dB
     
    # Method to predict the class label.
    def predict(self, x_new):
        test_preds = self.sigmoid(x_new, self.weight, self.bias)
        return self.get_clean_output(test_preds)


log_reg = LogisticRegressionCustom(0.1, 100)
log_reg.fit(X_train, y_train,X_test,y_test)
preds = log_reg.predict(X_test)
print("\nFinal Training accuracy: ",log_reg._accuracy(y_train.values, log_reg.predict(X_train)))
print("Final Testing accuracy:, ", log_reg._accuracy(y_test.values, preds))

# print classification report
print(classification_report(y_test, preds))

100%|██████████| 100/100 [00:29<00:00,  3.39it/s]



Final Training accuracy:  0.9988951569485733
Final Testing accuracy:,  0.9990028369989608
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71089
           1       0.81      0.49      0.61       113

    accuracy                           1.00     71202
   macro avg       0.90      0.74      0.80     71202
weighted avg       1.00      1.00      1.00     71202



In [ ]:
# logistic regression object
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
  
# train the model on train set
lr.fit(X_train, y_train)
  
predictions = lr.predict(X_test)
  
# print classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71089
           1       0.85      0.60      0.70       113

    accuracy                           1.00     71202
   macro avg       0.92      0.80      0.85     71202
weighted avg       1.00      1.00      1.00     71202



# Credit Card fraud prediction using original and custom model with SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(random_state = 42)
X_smote, y_smote = oversample.fit_resample(X_train, y_train)

In [ ]:
from tqdm import tqdm

class LogisticRegressionCustom:
    def __init__(self, lr, epochs):      
        self.lr = lr
        self.epochs = epochs
        self.weight = 0
        self.bias = 0
        self.loss = []
        self.accuracy_record = []
        self.weight_record = []
        self.bias_record = []
        self.epoch_record = []

    def get_clean_output(self, preds):
      threshold_prob = 0.5
      result = preds >= threshold_prob
      y_pred = np.zeros(result.shape[0])
      for i in range(len(y_pred)):
          if result[i] == True: 
              y_pred[i] = 1
          else:
              continue
                 
      return y_pred


    def _accuracy(self, target, predictions):
      accuracy = 0
      for i in range(len(predictions)):
        if (predictions[i] == target[i]):
          accuracy += 1

      return accuracy / len(target)

    # Sigmoid method
    def sigmoid(self, x, weight, bias):
        z = np.dot(x, weight) + bias
        return 1 / (1 + np.exp(-z))
     
    # method to calculate the binary cross entropy Loss
    def BCE_loss(self, preds, y):
        return (-y * np.log(preds) - (1 - y) * np.log(1 - preds)).mean()

    # method for calculating the gradients
    def gradient_descent(self, X, preds, target):
      d_loss_wrt_weight = (1 / X.shape[0]) * np.dot(X.T, (preds - target)) 
      d_loss_wrt_bias =  (1 / X.shape[0]) * np.sum(preds - target)
      return d_loss_wrt_weight, d_loss_wrt_bias
     
    def fit(self, X, y, X_val, y_val):
      self.weight = np.zeros(X.shape[1])
      # training set
      X = X.values
      y = y.values

      # Validation set
      X_val = X_val.values
      y_val = y_val.values

      # training
      for i in tqdm(range(self.epochs)):
        predictions = self.sigmoid(X, self.weight, self.bias)
        loss = self.BCE_loss(predictions, y)
        dW, dB = self.gradient_descent(X, predictions, y)
        
        # recording items
        self.loss.append(loss)
        # recording validation accuracy
        self.accuracy_record.append(self._accuracy(y_val, self.get_clean_output(self.sigmoid(X_val, self.weight, self.bias))))
        self.weight_record.append(self.weight)
        self.bias_record.append(self.bias)
        self.epoch_record.append(i)
 
        # Updating the weights
        self.weight -= self.lr * dW
        self.bias -= self.lr * dB
     
    # Method to predict the class label.
    def predict(self, x_new):
        test_preds = self.sigmoid(x_new, self.weight, self.bias)
        return self.get_clean_output(test_preds)


log_reg = LogisticRegressionCustom(0.1, 100)
log_reg.fit(X_smote, y_smote,X_test,y_test)
preds = log_reg.predict(X_test)
print("\nFinal Training accuracy: ",log_reg._accuracy(y_smote.values, log_reg.predict(X_smote)))
print("Final Testing accuracy:, ", log_reg._accuracy(y_test.values, preds))

# print classification report
print(classification_report(y_test, preds))

100%|██████████| 100/100 [00:29<00:00,  3.35it/s]



Final Training accuracy:  0.9417050453509422
Final Testing accuracy:,  0.9736243363950451
              precision    recall  f1-score   support

           0       1.00      0.97      0.99     71089
           1       0.05      0.91      0.10       113

    accuracy                           0.97     71202
   macro avg       0.53      0.94      0.54     71202
weighted avg       1.00      0.97      0.99     71202



In [ ]:
# logistic regression object
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
  
# train the model on train set
lr.fit(X_smote, y_smote)
  
predictions = lr.predict(X_test)
  
# print classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99     71089
           1       0.05      0.92      0.10       113

    accuracy                           0.97     71202
   macro avg       0.53      0.95      0.54     71202
weighted avg       1.00      0.97      0.99     71202



# Gradient Descent with Momentum with smote

In [ ]:
# this is a batch gradient descent as

from tqdm import tqdm

class LogisticRegressionCustom:
    def __init__(self, lr, epochs, momentum = 0.9):      
        self.lr = lr
        self.epochs = epochs
        self.weight = 0
        self.bias = 0
        self.loss = []
        self.accuracy_record = []
        self.weight_record = []
        self.bias_record = []
        self.epoch_record = []
        self.momentum = momentum
        self.vt_weight = 0
        self.vt_bias = 0

    def get_clean_output(self, preds):
      threshold_prob = 0.5
      result = preds >= threshold_prob
      y_pred = np.zeros(result.shape[0])
      for i in range(len(y_pred)):
          if result[i] == True: 
              y_pred[i] = 1
          else:
              continue
                 
      return y_pred


    def _accuracy(self, target, predictions):
      accuracy = 0
      for i in range(len(predictions)):
        if (predictions[i] == target[i]):
          accuracy += 1

      return accuracy / len(target)

    # Sigmoid method
    def sigmoid(self, x, weight, bias):
        z = np.dot(x, weight) + bias
        return 1 / (1 + np.exp(-z))
     
    # method to calculate the binary cross entropy Loss
    def BCE_loss(self, preds, y):
        return (-y * np.log(preds) - (1 - y) * np.log(1 - preds)).mean()

    # method for calculating the gradients
    def gradient_descent(self, X, preds, target):
      d_loss_wrt_weight = (1 / X.shape[0]) * np.dot(X.T, (preds - target)) 
      d_loss_wrt_bias =  (1 / X.shape[0]) * np.sum(preds - target)
      return d_loss_wrt_weight, d_loss_wrt_bias
     
    def fit(self, X, y, X_val, y_val):
      self.weight = np.zeros(X.shape[1])
      # training set
      X = X.values
      y = y.values

      # Validation set
      X_val = X_val.values
      y_val = y_val.values

      # training
      for i in tqdm(range(self.epochs)):
        predictions = self.sigmoid(X, self.weight, self.bias)
        loss = self.BCE_loss(predictions, y)
        dW, dB = self.gradient_descent(X, predictions, y)
        
        # recording items
        self.loss.append(loss)
        # recording validation accuracy
        # self.accuracy_record.append(self._accuracy(y_val, self.get_clean_output(self.sigmoid(X_val, self.weight, self.bias))))
        self.weight_record.append(self.weight)
        self.bias_record.append(self.bias)
        self.epoch_record.append(i)
 
        
        '''
        #compute gradients with respect to theta 
        gradients = compute_gradients(data, theta)
        #Update vt by equation (8)
        vt = gamma * vt + lr * gradients
        #update model parameter theta by equation (9)
        theta = theta - vt
        '''
        
        # Updating the weights using momentum
        vt_weight = self.momentum * self.vt_weight + self.lr * dW # ya self.lr = (1- momentum) ho
        self.weight -= vt_weight

        # updating the bias/intercept using momentum
        vt_bias = self.momentum * self.vt_bias + self.lr * dB
        self.bias -= vt_bias
     
    # Method to predict the class label.
    def predict(self, x_new):
        test_preds = self.sigmoid(x_new, self.weight, self.bias)
        return self.get_clean_output(test_preds)


log_reg = LogisticRegressionCustom(0.1, 100)
log_reg.fit(X_smote, y_smote,X_test,y_test)
preds = log_reg.predict(X_test)
print("\nFinal Training accuracy: ",log_reg._accuracy(y_smote.values, log_reg.predict(X_smote)))
print("Final Testing accuracy:, ", log_reg._accuracy(y_test.values, preds))

# print classification report
print(classification_report(y_test, preds))

100%|██████████| 100/100 [00:07<00:00, 12.50it/s]



Final Training accuracy:  0.9417050453509422
Final Testing accuracy:,  0.9736243363950451
              precision    recall  f1-score   support

           0       1.00      0.97      0.99     71089
           1       0.05      0.91      0.10       113

    accuracy                           0.97     71202
   macro avg       0.53      0.94      0.54     71202
weighted avg       1.00      0.97      0.99     71202



In [ ]:
%%time
# logistic regression object
lr = LogisticRegression()
  
# train the model on train set
lr.fit(X_smote, y_smote)
  
predictions = lr.predict(X_test)
  
# print classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99     71089
           1       0.05      0.92      0.10       113

    accuracy                           0.97     71202
   macro avg       0.53      0.95      0.54     71202
weighted avg       1.00      0.97      0.99     71202

CPU times: user 11.4 s, sys: 4.48 s, total: 15.9 s
Wall time: 13.5 s


# Adam optimizer with original and custom LR model

# Original working code (Don't touch this portion)

In [ ]:
# #### ----- ORIGINAL CODE ! DO NOT EDIT ------####

# from tqdm import tqdm

# class LogisticRegression:
#     def __init__(self, lr, epochs):      
#         self.lr = lr
#         self.epochs = epochs
#         self.weight = 0
#         self.bias = 0
#         self.loss = []
#         self.accuracy_record = []
#         self.weight_record = []
#         self.bias_record = []
#         self.epoch_record = []


#     def get_clean_output(self, preds):
#       threshold_prob = 0.5
#       result = preds >= threshold_prob
#       y_pred = np.zeros(result.shape[0])
#       for i in range(len(y_pred)):
#           if result[i] == True: 
#               y_pred[i] = 1
#           else:
#               continue
                 
#       return y_pred


#     def _accuracy(self, target, predictions):
#       accuracy = 0
#       for i in range(len(predictions)):
#         if (predictions[i] == target[i]):
#           accuracy += 1

#       return accuracy / len(target)

#     # using standarization for feature scaling
#     def standardize(self, X_tr):
#       for i in range(shape(X_tr)[1]):
#         X_tr[:,i] = (X_tr[:,i] - np.mean(X_tr[:,i]))/np.std(X_tr[:,i])
#       return X_tr

#     # Sigmoid method
#     def sigmoid(self, x, weight, bias):
#         z = np.dot(x, weight) + bias
#         return 1 / (1 + np.exp(-z))
     
#     # method to calculate the binary cross entropy Loss
#     def BCE_loss(self, preds, y):
#         return (-y * np.log(preds) - (1 - y) * np.log(1 - preds)).mean()

#     # method for calculating the gradients
#     def gradient_descent(self, X, preds, target):
#       d_loss_wrt_weight = (1 / X.shape[0]) * np.dot(X.T, (preds - target)) 
#       d_loss_wrt_bias =  (1 / X.shape[0]) * np.sum(preds - target)
#       return d_loss_wrt_weight, d_loss_wrt_bias
     
#     def fit(self, X, y, X_val, y_val):
#       self.weight = np.zeros(X.shape[1])
#       # training set
#       X = self.standardize(X.values)
#       y = y.values

#       # Validation set
#       X_val = self.standardize(X_val.values)
#       y_val = y_val.values

#       # training
#       for i in tqdm(range(self.epochs)):
#         predictions = self.sigmoid(X, self.weight, self.bias)
#         loss = self.BCE_loss(predictions, y)
#         dW, dB = self.gradient_descent(X, predictions, y)
        
#         # recording items
#         self.loss.append(loss)
#         # recording validation accuracy
#         self.accuracy_record.append(self._accuracy(y_val, self.get_clean_output(self.sigmoid(X_val, self.weight, self.bias))))
#         self.weight_record.append(self.weight)
#         self.bias_record.append(self.bias)
#         self.epoch_record.append(i)
 
#         # Updating the weights
#         self.weight -= self.lr * dW
#         self.bias -= self.lr * dB


#       print('Attempt Sucessful')
     
#     # Method to predict the class label.
#     def predict(self, x_new):
#         test_preds = self.sigmoid(x_new, self.weight, self.bias)
#         return self.get_clean_output(test_preds)